In [20]:
import subprocess
from rag import load_model, load_data, inference
from IPython.display import Markdown, display
from fpdf import FPDF



In [21]:
harvey, harvey_embed = load_model()
litt, litt_embed = load_model()
judge, judge_embed = load_model()

harvey_engine = load_data(harvey, harvey_embed, 'prosecutor')
litt_engine = load_data(litt, litt_embed, 'defense')
judge_engine = load_data(judge, judge_embed, 'judge')

Generating embeddings: 100%|██████████| 8/8 [00:08<00:00,  1.06s/it]


prosecutor


Generating embeddings: 100%|██████████| 8/8 [00:04<00:00,  1.61it/s]


defense


Generating embeddings: 100%|██████████| 8/8 [00:05<00:00,  1.41it/s]

judge


In [ ]:

import os
from pypdf import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    number_of_pages = len(reader.pages)
    text = ""
    for i in range(number_of_pages):
        page = reader.pages[i]
        text += page.extract_text()
    return text

upload_dir = './uploads'
for filename in os.listdir(upload_dir):
    if filename.endswith('.pdf'):
        file_path = os.path.join(upload_dir, filename)
        initial_case = extract_text_from_pdf(file_path)
        # print(f"Text from {filename}:\n{initial_text}\n")

In [25]:
epochs = 5
num_conversations = 5



# initial_case = """Title: Complaint for Unlawful Fees Imposed by ABC Property Management LLC
# Case Number: 2024-CV-123456
# Parties:
# Plaintiff (Tenant): John Doe, residing at 123 Main Street, Apt 2B, Chicago, IL 60601.
# Defendant (Landlord): ABC Property Management LLC, registered office at 456 Market Avenue,
# Chicago, IL 60602.
# Introduction: This complaint is brought by the Plaintiff, John Doe, against the Defendant, ABC
# Property Management LLC, for the imposition of unnecessary and unlawful fees in connection with a
# rental agreement.
# Facts of the Case:
# The Plaintiff entered into a rental agreement with the Defendant on January 1, 2023.
# The lease stipulated a monthly rent of $1,200 with no additional mandatory fees beyond utilities.
# On February 1, 2024, the Defendant issued an invoice that included a $250 "administrative
# maintenance fee."
# The Plaintiff disputed the fee, citing that it was not included in the lease agreement.
# The Defendant refused to remove the charge and threatened eviction if payment was not made.
# Claims for Relief:
# Breach of Contract: The Defendant has violated the terms of the lease by imposing unauthorized fees.
# Violation of Illinois Tenant Protection Act: The fee constitutes an unlawful charge under state
# regulations.
# Relief Sought:
# Refund of $250 to the Plaintiff.
# Compensation for attorney's fees incurred in pursuing this claim.
# Any additional relief deemed just and proper by the court."""

for conversation in range(num_conversations):
    with open(f"conversations/conversation_{conversation+1}.txt", "w") as pdf:
        for epoch in range(epochs):
            prosecutor_prompt = f"Use your knowledge of the given documents and Illinois State Law to create the best rebuttal claim to the case argument. Your response should only be about one part of the case argument and should not contain any reasoning, only your claim with a brief citation and explanation. Any claim should be restricted to the law of the state of Illinois: {initial_case}"
            prosecutor_response = inference(litt_engine, prosecutor_prompt)
            pdf.write(str(prosecutor_response))
            pdf.write("\n\n")
            display(Markdown(str(prosecutor_response)))

            judge_prompt = f"Look through the argument and using prior knowledge, determine if the argument is valid or invalid, and make sure this reasoning is not in your response. If it is invalid, respond with only the word 'invalid'. If it is valid, respond with only the word 'valid': {prosecutor_response}"
            judge_response = inference(judge_engine, judge_prompt)
            display(Markdown(str(judge_response)))

            if (str(judge_response) == "invalid"):
                break

            harvey_prompt = f"Use your knowledge of the given documents and Illinois State Law to create a strong counterclaim that directly addresses the argument and your response should only be the counterclaim with a brief citation and explanation. Any argument should be restricted to the law of the state of Illinois: {prosecutor_response}"
            harvey_response = inference(harvey_engine, harvey_prompt)
            pdf.write(str(harvey_response))
            pdf.write("\n\n")
            display(Markdown(str(harvey_response)))
            initial_case = harvey_response


print(initial_case)

In response to the Defendant's Claim of Breach of Contract, I move that the Court find that the Defendant's imposition of a $250 "administrative maintenance fee" without prior written notice or approval from the Plaintiff as required under Illinois law constitutes a breach of contract.

ILCS 765/14(a)(2) states that "[a] landlord shall not charge a tenant any fee, other than utilities and reasonable charges for repairs and maintenance authorized by this Act."

Citation: ILCS 765/14(a)(2).

valid.

The Defendant's Claim of Breach of Contract is hereby countered with the following argument:

The Plaintiff's claim that the Defendant's imposition of a $250 "administrative maintenance fee" without prior written notice or approval constitutes a breach of contract. I move that the Court find that the Defendant's actions are, in fact, permitted under Illinois law as a reasonable charge for repairs and maintenance authorized by the Illinois Residential Landlord-Tenant Act (ILSRLTA). Specifically, ILCS 765/14(a)(2) authorizes landlords to charge tenants fees for certain authorized repairs and maintenance, and it is hereby argued that the $250 fee in question falls within this exception.

Citation: ILCS 765/14(a)(2); Rendall v. Smith, 336 Ill. App. 3d 551 (2001) (upholding a landlord's authority to charge a tenant for authorized repairs and maintenance).

This counterclaim seeks to demonstrate that the Defendant's actions do not constitute a breach of contract, but rather a lawful exercise of its rights under Illinois law.

The Plaintiff's claim that the Defendant's imposition of a $250 "administrative maintenance fee" without prior written notice or approval constitutes a breach of contract. I move that the Court find that the Defendant's actions are, in fact, permitted under Illinois law as a reasonable charge for repairs and maintenance authorized by the Illinois Residential Landlord-Tenant Act (ILSRLTA). Specifically, ILCS 765/14(a)(2) authorizes landlords to charge tenants fees for certain authorized repairs and maintenance, and it is hereby argued that the $250 fee in question falls within this exception.

Citation: ILCS 765/14(a)(2); Rendall v. Smith, 336 Ill. App. 3d 551 (2001) (upholding a landlord's authority to charge a tenant for authorized repairs and maintenance).

This counterclaim seeks to demonstrate that the Defendant's actions do not constitute a breach of contract, but rather a lawful exercise of its rights under Illinois law.

Additionally, I would like to bring to the Court's attention that pursuant to ILCS 765/14(a)(2), such fees are subject to reasonable standards and limitations. Therefore, the Plaintiff shall also argue that the Defendant's $250 fee is unreasonable and excessive, as it exceeds the reasonable standards set forth in the Act.

Citation: ILCS 765/14(a)(2) (requiring fees for authorized repairs and maintenance to be reasonable).

valid.

...

valid

Counterclaim:

The Plaintiff's assertion that the $250 administrative maintenance fee is a breach of contract is countered as follows:

The Plaintiff's failure to obtain landlord approval prior to incurring expenses does not render the fee a breach of contract, but rather an invalid expense under Section 5(b) of the Illinois Residential Landlord Tenant Act (ILCS 765/5(b)). The Plaintiff's actions do not constitute a breach, but rather a violation of the statute. Therefore, the Defendant is not liable for the $250 administrative maintenance fee.

Citation: ILCS 765/5(b)

Explanation: By failing to seek approval, the Plaintiff did not breach any contractual obligation with the Defendant, and therefore, the fee is not recoverable as a breach of contract.

The Defendant's claim that the Plaintiff failed to provide proper notice of termination under Section 9(a) of the Illinois Residential Landlord Tenant Act (ILCS 765/9(a)) is without merit. The Plaintiff's failure to comply with the statutory requirement for a three-day written notice of termination prior to initiating eviction proceedings does not render the eviction proceeding invalid, but rather constitutes a procedural default that can be waived by the court upon request.

Citation: ILCS 765/9(a)

Explanation: By providing any form of written notice to terminate the tenancy, the Plaintiff has complied with the statutory requirement, and the Defendant's failure to do so does not vitiate the eviction proceeding.

valid

KeyboardInterrupt: 

In [16]:
initial_case_str = str(initial_case)
display(Markdown(initial_case))

TypeError: Markdown expects text, not <IPython.core.display.Markdown object>